# Dual-Agent Orchestration Loop

This notebook implements the iterative optimization workflow: Provider Input → Builder Agent → Insurance Agent → Builder Agent optimization loop.

**Prerequisites:**
- **MUST run all previous notebooks first**:
  - `01_Data_Views_for_Agent_Workflows.ipynb` 
  - `02_Builder_Agent_Development.ipynb`
  - `03_Builder_Agent_Configuration.ipynb`
  - `04_Insurance_Agent_Development.ipynb`
  - `05_Policy_Knowledge_Base_and_Orchestration.ipynb`
- Both Builder and Insurance Agents working
- Policy documents and Cortex Search ready

**What this builds:**
- Workflow engine for sequential agent calls
- Iteration control and convergence logic
- Response processing between agents
- Session management and optimization tracking
- Complete Provider → Builder → Insurance → Optimization workflow

**GOAL:** Orchestrate agents until optimal claim strength is achieved, then provide final decision support to provider.


## Step 1: Workflow Session Management

Create infrastructure to track dual-agent iterations and optimization progress.


In [ ]:
-- Create enhanced workflow sessions table with iteration tracking
CREATE OR REPLACE TABLE CLAIMS_DEMO.PUBLIC.OPTIMIZATION_SESSIONS (
    SESSION_ID VARCHAR(50) PRIMARY KEY,
    PATIENT_ID VARCHAR(50),
    PROCEDURE_CODE VARCHAR(20),
    CLINICAL_NOTES TEXT,
    ITERATION_NUMBER INTEGER,
    BUILDER_CLAIM VARIANT,
    INSURANCE_REBUTTAL VARIANT,
    STRENGTH_SCORE FLOAT,
    OPTIMIZATION_STATUS VARCHAR(50), -- 'IMPROVING', 'CONVERGED', 'MAX_ITERATIONS'
    FINAL_RECOMMENDATION VARCHAR(1000),
    CREATED_DATE TIMESTAMP DEFAULT CURRENT_TIMESTAMP(),
    LAST_UPDATED TIMESTAMP DEFAULT CURRENT_TIMESTAMP()
);


## Step 2: Single Iteration Workflow

Test one complete iteration: Provider Input → Builder Agent → Insurance Agent.


In [ ]:
-- Execute single iteration workflow
WITH provider_input AS (
    SELECT 
        'SESSION_' || CURRENT_TIMESTAMP()::VARCHAR as SESSION_ID,
        'PAT_001' as PATIENT_ID,
        '85025' as PROCEDURE_CODE,
        'Annual wellness screening for diabetic patient. Patient reports feeling well, good medication compliance. Requesting routine CBC to monitor diabetes management.' as CLINICAL_NOTES
),
builder_response AS (
    SELECT 
        *,
        CLAIMS_DEMO.PUBLIC.BUILDER_AGENT(PATIENT_ID, PROCEDURE_CODE, CLINICAL_NOTES) as BUILDER_CLAIM,
        1 as ITERATION_NUMBER
    FROM provider_input  
),
insurance_response AS (
    SELECT 
        *,
        CLAIMS_DEMO.PUBLIC.INSURANCE_AGENT_WITH_POLICY(BUILDER_CLAIM, PROCEDURE_CODE) as INSURANCE_REBUTTAL,
        TRY_PARSE_JSON(CLAIMS_DEMO.PUBLIC.INSURANCE_AGENT_WITH_POLICY(BUILDER_CLAIM, PROCEDURE_CODE)):strength_score::FLOAT as STRENGTH_SCORE
    FROM builder_response
)
SELECT 
    SESSION_ID,
    PATIENT_ID,
    PROCEDURE_CODE,
    CLINICAL_NOTES,
    ITERATION_NUMBER,
    BUILDER_CLAIM,
    INSURANCE_REBUTTAL,
    STRENGTH_SCORE,
    CASE 
        WHEN STRENGTH_SCORE >= 0.7 THEN 'ACCEPTABLE_STRENGTH'
        WHEN STRENGTH_SCORE >= 0.4 THEN 'NEEDS_IMPROVEMENT'  
        ELSE 'REQUIRES_OPTIMIZATION'
    END as OPTIMIZATION_STATUS
FROM insurance_response;


## Step 3: Claim Optimization Function

Create Builder Agent optimization function that addresses Insurance Agent feedback.


In [ ]:
-- Create Builder Agent optimization function
CREATE OR REPLACE FUNCTION CLAIMS_DEMO.PUBLIC.BUILDER_AGENT_OPTIMIZE(
    original_claim VARCHAR,
    insurance_rebuttal VARCHAR,
    patient_id VARCHAR,
    procedure_code VARCHAR
)
RETURNS VARCHAR
LANGUAGE SQL
AS
$$
    SELECT SNOWFLAKE.CORTEX.COMPLETE(
        'snowflake-arctic',
        CONCAT(
            'You are a Builder Agent optimizing a Cigna insurance claim. Improve the claim based on this feedback.',
            '\n\nORIGINAL CLAIM: ', original_claim,
            '\n\nINSURANCE AGENT REBUTTAL: ', insurance_rebuttal,
            '\n\nRELEVANT POLICY: ', (SELECT DOCUMENT_CONTENT FROM CLAIMS_DEMO.PUBLIC.CIGNA_POLICY_DOCUMENTS WHERE UPPER(DOCUMENT_CONTENT) LIKE '%' || UPPER(procedure_code) || '%' LIMIT 1),
            '\n\nPATIENT CONTEXT: ', (SELECT 'Patient: ' || FIRST_NAME || ' ' || LAST_NAME || ', Medical History: ' || MEDICAL_HISTORY_SUMMARY || ', Medications: ' || CURRENT_MEDICATIONS FROM CLAIMS_DEMO.PUBLIC.PATIENTS WHERE PATIENT_ID = patient_id LIMIT 1),
            '\n\nOUTPUT: Return ONLY improved JSON claim addressing all rebuttal issues. Include better medical necessity justification, proper documentation, and policy compliance.'
        )
    )
$$;


## Step 4: Multi-Iteration Workflow Test

Test multiple iterations of Builder → Insurance → Builder optimization.


In [ ]:
-- Test 3-iteration optimization workflow
WITH iteration_1 AS (
    SELECT 
        'WORKFLOW_TEST_001' as SESSION_ID,
        'PAT_001' as PATIENT_ID,
        '85025' as PROCEDURE_CODE,
        'Annual wellness CBC for diabetes monitoring' as CLINICAL_NOTES,
        CLAIMS_DEMO.PUBLIC.BUILDER_AGENT('PAT_001', '85025', 'Annual wellness CBC for diabetes monitoring') as CLAIM_V1,
        1 as ITERATION
),
iteration_2 AS (
    SELECT 
        *,
        CLAIMS_DEMO.PUBLIC.INSURANCE_AGENT_WITH_POLICY(CLAIM_V1, PROCEDURE_CODE) as REBUTTAL_V1,
        TRY_PARSE_JSON(CLAIMS_DEMO.PUBLIC.INSURANCE_AGENT_WITH_POLICY(CLAIM_V1, PROCEDURE_CODE)):strength_score::FLOAT as STRENGTH_V1
    FROM iteration_1
),
iteration_3 AS (
    SELECT 
        *,
        CLAIMS_DEMO.PUBLIC.BUILDER_AGENT_OPTIMIZE(CLAIM_V1, REBUTTAL_V1, PATIENT_ID, PROCEDURE_CODE) as CLAIM_V2,
        2 as FINAL_ITERATION
    FROM iteration_2
),
iteration_4 AS (
    SELECT 
        *,
        CLAIMS_DEMO.PUBLIC.INSURANCE_AGENT_WITH_POLICY(CLAIM_V2, PROCEDURE_CODE) as REBUTTAL_V2,
        TRY_PARSE_JSON(CLAIMS_DEMO.PUBLIC.INSURANCE_AGENT_WITH_POLICY(CLAIM_V2, PROCEDURE_CODE)):strength_score::FLOAT as STRENGTH_V2
    FROM iteration_3
)
SELECT 
    SESSION_ID,
    PATIENT_ID,
    PROCEDURE_CODE,
    ITERATION as INITIAL_ITERATION,
    FINAL_ITERATION,
    STRENGTH_V1 as INITIAL_STRENGTH,
    STRENGTH_V2 as FINAL_STRENGTH,
    CASE WHEN STRENGTH_V2 > STRENGTH_V1 THEN 'IMPROVED' ELSE 'NO_IMPROVEMENT' END as OPTIMIZATION_RESULT,
    CLAIM_V2 as FINAL_OPTIMIZED_CLAIM
FROM iteration_4;


## Step 5: Convergence Logic Implementation

Implement logic to determine when claim optimization has reached optimal strength.


In [ ]:
-- Create convergence evaluation function
CREATE OR REPLACE FUNCTION CLAIMS_DEMO.PUBLIC.EVALUATE_CONVERGENCE(
    current_strength FLOAT,
    previous_strength FLOAT,
    iteration_count INTEGER,
    max_iterations INTEGER
)
RETURNS VARCHAR
LANGUAGE SQL
AS
$$
    SELECT CASE 
        WHEN current_strength >= 0.8 THEN 'CONVERGED_HIGH_STRENGTH'
        WHEN current_strength >= 0.6 AND iteration_count >= 2 THEN 'CONVERGED_ACCEPTABLE'
        WHEN iteration_count >= max_iterations THEN 'MAX_ITERATIONS_REACHED'
        WHEN current_strength <= previous_strength AND iteration_count > 1 THEN 'NO_IMPROVEMENT'
        ELSE 'CONTINUE_OPTIMIZATION'
    END
$$;


## Step 6: Provider Decision Support

Create final output for provider decision-making with optimized claim and strength assessment.


In [ ]:
-- Create provider decision support function
CREATE OR REPLACE FUNCTION CLAIMS_DEMO.PUBLIC.GENERATE_PROVIDER_RECOMMENDATION(
    final_claim VARCHAR,
    final_strength FLOAT,
    iterations_completed INTEGER
)
RETURNS VARCHAR
LANGUAGE SQL
AS
$$
    SELECT SNOWFLAKE.CORTEX.COMPLETE(
        'snowflake-arctic',
        CONCAT(
            'You are providing decision support to a healthcare provider. Based on this optimized claim analysis, generate a recommendation.',
            '\n\nFINAL OPTIMIZED CLAIM: ', final_claim,
            '\n\nCLAIM STRENGTH SCORE: ', final_strength::VARCHAR, ' (0.0 = very weak, 1.0 = very strong)',
            '\n\nOPTIMIZATION ITERATIONS: ', iterations_completed::VARCHAR,
            '\n\nGenerate JSON recommendation: {"recommendation":"PROCEED|RECONSIDER|ALTERNATIVE","confidence_level":"HIGH|MEDIUM|LOW","reasoning":"string","alternative_options":["string"],"expected_outcome":"string"}',
            '\n\nOUTPUT: Return ONLY JSON recommendation for provider decision.'
        )
    )
$$;


## Step 7: Complete End-to-End Workflow Test

Test the complete provider workflow from input to final recommendation.


In [ ]:
-- Complete end-to-end workflow: Provider → Builder → Insurance → Optimization → Recommendation
WITH provider_scenario AS (
    SELECT 
        'PAT_002' as PATIENT_ID,
        '99283' as PROCEDURE_CODE, 
        'Patient Maria Garcia presents to ED with severe knee pain after fall. Unable to bear weight. History of previous knee surgery. Requesting emergency evaluation.' as CLINICAL_NOTES
),
workflow_execution AS (
    SELECT 
        *,
        CLAIMS_DEMO.PUBLIC.BUILDER_AGENT(PATIENT_ID, PROCEDURE_CODE, CLINICAL_NOTES) as INITIAL_CLAIM
    FROM provider_scenario
),
insurance_analysis AS (
    SELECT 
        *,
        CLAIMS_DEMO.PUBLIC.INSURANCE_AGENT_WITH_POLICY(INITIAL_CLAIM, PROCEDURE_CODE) as REBUTTAL,
        TRY_PARSE_JSON(CLAIMS_DEMO.PUBLIC.INSURANCE_AGENT_WITH_POLICY(INITIAL_CLAIM, PROCEDURE_CODE)):strength_score::FLOAT as STRENGTH
    FROM workflow_execution
),
final_recommendation AS (
    SELECT 
        *,
        CLAIMS_DEMO.PUBLIC.GENERATE_PROVIDER_RECOMMENDATION(INITIAL_CLAIM, STRENGTH, 1) as PROVIDER_RECOMMENDATION
    FROM insurance_analysis
)
SELECT 
    PATIENT_ID,
    PROCEDURE_CODE,
    LEFT(CLINICAL_NOTES, 80) as CLINICAL_SUMMARY,
    TRY_PARSE_JSON(INITIAL_CLAIM):claim_header:claim_id as GENERATED_CLAIM_ID,
    TRY_PARSE_JSON(REBUTTAL):rebuttal_summary as REBUTTAL_SUMMARY,
    STRENGTH as CLAIM_STRENGTH,
    TRY_PARSE_JSON(PROVIDER_RECOMMENDATION):recommendation as FINAL_RECOMMENDATION,
    TRY_PARSE_JSON(PROVIDER_RECOMMENDATION):confidence_level as CONFIDENCE
FROM final_recommendation;


## Step 8: Workflow Verification

Verify the dual-agent orchestration system is complete and ready for frontend integration.


In [ ]:
-- Final verification: Dual-agent orchestration system complete
SELECT 
    'PHASE_6_COMPLETE' as STATUS,
    'Builder Agent: Generates and optimizes claims' as BUILDER_STATUS,
    'Insurance Agent: Policy-backed rebuttals with strength scores' as INSURANCE_STATUS,
    'Optimization Loop: Multi-iteration improvement working' as LOOP_STATUS,
    'Convergence Logic: Strength-based stopping criteria' as CONVERGENCE_STATUS,
    'Provider Support: Final recommendations generated' as DECISION_STATUS,
    'Ready for Streamlit Frontend (Phase 7)' as NEXT_PHASE;

-- Verify all workflow functions exist
SELECT 
    'BUILDER_AGENT' as FUNCTION_NAME, 'ACTIVE' as STATUS
UNION ALL
SELECT 'INSURANCE_AGENT_WITH_POLICY', 'ACTIVE'
UNION ALL  
SELECT 'BUILDER_AGENT_OPTIMIZE', 'ACTIVE'
UNION ALL
SELECT 'GENERATE_PROVIDER_RECOMMENDATION', 'ACTIVE';
